In [3]:
import json
import types
import requests

In [4]:
class Namespace:
    def __init__(self, adict):
        self.__dict__.update(adict)

In [5]:
#base_link = "http://localhost:8080/api/v1/"
base_link = "http://192.168.0.218:8080/api/v1/"

In [6]:
links = {
    "position":Namespace({
        "all":"position/all",
        "all_active":"position/all/active",
        "single":"position/"
    }),
    "employee":Namespace({
        "all":"employee/all",
        "single":"employee/"
    }),
    "task":Namespace({
        "all":"task/all",
        "single":"task/"
    }),
    "taskrole":Namespace({
        "all":"taskrole/all",
        "single":"taskrole/",
        "position":"taskrole/position/"
    }),
    "logbook":Namespace({
        "all":"logbook/all",
        "range":"logbook/range/", #{s}/{e}
        "prange":"logbook/range/",#{p}/{s}/{e}
        "single":"logbook/"
    }),
    "absence":Namespace({
        "all":"absence/all",
        "range":"absence/range/", #{s}/{e}
        "prange":"absence/range/",#{p}/{s}/{e}
        "single":"absence/"
    })
};links_ns = Namespace(links)

In [7]:
def get_direction_name(p):
    p = Namespace(p)
    if p.service is not None:
        subdiv = p.service["subdivision"]
        subdiv = Namespace(subdiv)
        if subdiv.direction is not None:
            return subdiv.direction["shortName"]
        else: return subdiv.shortName
    else:
        div = p.subdivision["direction"]
        if div is not None:
            return div["shortName"]
        else: return p.subdivision["shortName"]
        
def get_service_name(p):
    p = Namespace(p)
    if p.service is not None:
        return p.service["name"]
    else:
        return "AUCUN"

In [8]:
def get_employee_list_bio_data():
    emp_data_list = list()
    positions = requests.get(base_link+links_ns.position.all_active).json()
    for p in positions:
        emp_data = dict()
        emp_data["position_id"] = p["positionId"]
        emp_data["employee_id"] = p["employee"]["employeeId"]
        emp_data["fullname"] = p["employee"]["lastName"] +" "+ p["employee"]["firstName"].strip() 
        emp_data["matricula"] = p["employee"]["matricule"]
        emp_data["rolename"] = p["role"]["name"]
        emp_data["service"] = get_service_name(p)
        emp_data["direction"] = get_direction_name(p)
        emp_data["email"] = p["employee"]["email"]
        emp_data["contact"] = p["employee"]["contact"]
        emp_data["location"] = p["location"]
        emp_data_list.append(emp_data)
    return emp_data_list

In [9]:
#get_employee_list_bio_data()
emp_list = get_employee_list_bio_data()

In [10]:
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta

#date_string = "2023-03-01" "%Y-%m-%d"

def extract_year_from_date(date_string:str):
    date_obj = datetime.strptime(date_string, "%Y-%m-%d")
    return date_obj.year

def extract_month_from_date(date_string:str):
    date_obj = datetime.strptime(date_string, "%Y-%m-%d")
    return date_obj.month

def extract_workdays_in_month(year, month):
    start_date = dt.date(year, month, 1)
    end_date = dt.date(year, month+1, 1) - dt.timedelta(days=1)
    return pd.bdate_range(start_date, end_date, freq='C').size

In [11]:
def compute_time_diff_in_minutes(checkin:str):
    time1 = '08:10:00'
    time2 = checkin
    dt1 = datetime.strptime(time1, '%H:%M:%S')
    dt2 = datetime.strptime(time2, '%H:%M:%S')
    time_diff = dt1 - dt2
    minutes_diff = time_diff.total_seconds() / 60.0
    return minutes_diff

def count_late_occurence(minuteDiffs:list):
    occurences = [1 for t in minuteDiffs if t<0]
    return len(occurences)

def count_no_log_occurence(nb_log:int, date_string:str):
    n_work_days = compute_word_days_in_month(date_string)
    return n_work_days - nb_log

def compute_word_days_in_month(date_string:str):
    year = extract_year_from_date(date_string)
    month = extract_month_from_date(date_string)
    return extract_workdays_in_month(year, month)

def generate_daterange_link(link:links_ns, p, s, e):
    """Link namespace"""
    return base_link+link.prange+ str(p) +"/"+ s +"/"+ e

def generate_position_req_link(link:links_ns, p):
    """Link namespace"""
    return base_link + link.position + str(p)

In [12]:
def get_employee_list_checkin_data(emp_list, start_date:str, end_date:str):
    log_data_list = []
    for emp in emp_list:

        logd = dict()

        e = Namespace(emp)
        pid = e.position_id

        lc = generate_daterange_link(links_ns.logbook, pid, start_date, end_date)
        la = generate_daterange_link(links_ns.absence, pid, start_date, end_date)

        reqa = requests.get(la)
        reqc = requests.get(lc)

        loga = reqa.json()
        logc = reqc.json()

        #logd["position_id"] = pid
        logd["checkins"] = len(logc)
        logd["no_checkins"] = count_no_log_occurence(len(logc), start_date)
        logd["late_arrivals"] = count_late_occurence([compute_time_diff_in_minutes(c["checkinTime"]) for c in logc])
        logd["absences"] = len(loga)
        logd["abs_justified"] = len([1 for a in loga if a["justified"]=="yes"])

        log_data_list.append(logd)
        emp["logs"] = logd
    return emp_list, log_data_list

In [13]:
emp_list_1 = get_employee_list_checkin_data(emp_list,"2023-04-01","2023-04-30")[0]

In [14]:
def compute_nb_finished_tasks(taskrole):
    finished = [x["task"]["status"] for x in taskrole if x["task"]["status"]=="FINISHED"]
    return len(finished)

def compute_progress(finished:int, nb:int):
    return finished/nb*100

In [15]:
def get_employee_list_task_data(emp_list):
    task_data_list = []
    for emp in emp_list:

        taskd = dict()

        e = Namespace(emp)
        pid = e.position_id

        l = generate_position_req_link(links_ns.taskrole, pid)

        d = requests.get(l).json()

        if len(d) == 0:
            task_data = {"nb_task":0,"nb_finished":0,"progress":0}
        else:
            nb = len(d)
            finished = compute_nb_finished_tasks(d)
            progress = compute_progress(finished, nb)

            task_data = {"nb_task":nb, "nb_finished": finished, "progress":progress}

        task_data_list.append(task_data)
        emp["tasks"] = task_data
    return emp_list, task_data_list

In [16]:
get_employee_list_task_data(emp_list_1)[0]

[{'position_id': 1,
  'employee_id': 1,
  'fullname': 'ANGOUA YOBOUATH GUY CHARLES',
  'matricula': '1080720A',
  'rolename': 'SOUS-DIRECTEUR',
  'service': 'AUCUN',
  'direction': 'DSI',
  'email': 'guy-charles.angoua@arti.ci',
  'contact': '',
  'location': 'ABIDJAN',
  'logs': {'checkins': 1,
   'no_checkins': 19,
   'late_arrivals': 0,
   'absences': 3,
   'abs_justified': 3},
  'tasks': {'nb_task': 0, 'nb_finished': 0, 'progress': 0}},
 {'position_id': 2,
  'employee_id': 2,
  'fullname': 'ANOH AMA LISETTE DESIREE Epse AFFANOU',
  'matricula': '2270921A',
  'rolename': 'AGENT',
  'service': 'SERVICE DE LA COMMUNICATION',
  'direction': 'DCP',
  'email': 'lisette.affanou@arti.ci',
  'contact': '0799852917',
  'location': 'ABIDJAN',
  'logs': {'checkins': 4,
   'no_checkins': 16,
   'late_arrivals': 2,
   'absences': 1,
   'abs_justified': 1},
  'tasks': {'nb_task': 0, 'nb_finished': 0, 'progress': 0}},
 {'position_id': 3,
  'employee_id': 3,
  'fullname': 'BAILLY ERIC THEODORE',
  